In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pygam as pg
from sklearn.model_selection import train_test_split
from pygam import LogisticGAM, s, te


In [4]:
#Seeing all columns
pd.set_option('display.max_columns', None)

# Loading in cleaned Dataset with sweetspot feature
pitches_ss = pd.read_csv("BIP_with_GIDP_SweetSpot.csv")


In [5]:
pitches_ss.head()

,gameid,GameDate,ab,pitchnum,inning,teambat,balls,strikes,outs,visscore,homscore,pitcher,pitcherthrows,batter,batterside,pitchresult,pitchresult_desc,eventtype,metrics_pitching_position,pitchname,pitchname_desc,spinrate,relspeed,horzbreak,inducedvertbreak,vertbreak,platelocside,platelocheight,ExitSpeed,VExitAngle,HExitAngle,initposx,initposy,initposz,hometeam_id,Home,awayteam_id,Visitor,venue_id,venue_name,video_broadcast,GIDP_SweetSpot
0,2022/04/07/cinmlb-atlmlb-1,08:00.0,3,3,1,0,2,0,2,0,0,"Fried, Max",L,"Pham, Tommy",R,X,Hit Into Play - Out(s),field_out,Stretch,FF,Four-seam FB,2176.828,95.56983,2.139670,11.539330,-18.21491,-1.113873,2.525816,55.31276,-42.227200,-0.448136,0.882,50,6.069,144,Atlanta Braves,113,Cincinnati Reds,4705,Truist Park,https://sporty-clips.mlb.com/R3hWZExfV0ZRVkV3d...,0
1,2022/04/07/cinmlb-atlmlb-1,08:00.0,6,3,1,1,1,1,2,0,0,"Mahle, Tyler",R,"Riley, Austin",R,D,Hit Into Play - No Out(s),single,Windup,FF,Four-seam FB,2360.456,93.33314,9.302727,16.755390,-14.57288,-0.499844,2.849501,91.72955,-0.069351,12.199050,-2.658,50,5.441,144,Atlanta Braves,113,Cincinnati Reds,4705,Truist Park,https://sporty-clips.mlb.com/R3hWZExfV0ZRVkV3d...,0
2,2022/04/07/cinmlb-atlmlb-1,08:00.0,7,2,1,1,0,1,2,0,0,"Mahle, Tyler",R,"Ozuna, Marcell",R,X,Hit Into Play - Out(s),field_out,Stretch,SL,Slider,2663.398,84.64002,-4.788012,1.895149,-35.77917,-0.035831,2.105335,100.97140,36.611920,-17.393650,-2.744,50,5.532,144,Atlanta Braves,113,Cincinnati Reds,4705,Truist Park,https://sporty-clips.mlb.com/R3hWZExfV0ZRVkV3d...,0
3,2022/04/07/cinmlb-atlmlb-1,08:00.0,10,3,2,0,1,1,1,0,0,"Fried, Max",L,"Senzel, Nick",R,D,Hit Into Play - No Out(s),single,Stretch,SL,Slider,2307.948,89.99753,4.627371,3.820850,-29.52298,0.035648,2.418733,98.19635,8.123025,-3.759704,1.352,50,6.026,144,Atlanta Braves,113,Cincinnati Reds,4705,Truist Park,https://sporty-clips.mlb.com/R3hWZExfV0ZRVkV3d...,0
4,2022/04/07/cinmlb-atlmlb-1,08:00.0,11,3,2,0,2,0,1,0,0,"Fried, Max",L,"Moustakas, Mike",L,X,Hit Into Play - Out(s),field_out,Stretch,FF,Four-seam FB,2167.002,95.81812,0.946951,14.476550,-15.27192,-0.957702,2.325284,69.13850,64.647930,-1.996992,0.921,50,6.013,144,Atlanta Braves,113,Cincinnati Reds,4705,Truist Park,https://sporty-clips.mlb.com/R3hWZExfV0ZRVkV3d...,0


Recreating existing GAMs

In [6]:
pitches_ss = pitches_ss.dropna()

In [7]:
# Create four different subsets based on pitcher-batter handedness
RHP_RHH = pitches_ss[(pitches_ss['pitcherthrows'] == 'R') & (pitches_ss['batterside'] == 'R')]
RHP_LHH = pitches_ss[(pitches_ss['pitcherthrows'] == 'R') & (pitches_ss['batterside'] == 'L')]
LHP_RHH = pitches_ss[(pitches_ss['pitcherthrows'] == 'L') & (pitches_ss['batterside'] ==  'R')]
LHP_LHH = pitches_ss[(pitches_ss['pitcherthrows'] == 'L') & (pitches_ss['batterside'] == 'L')]

In [9]:
#Recreating Gams Jay made in analysis.ipynb using 

#RHP V RHH
X_RR = RHP_RHH[['spinrate', 'relspeed', 'horzbreak', 'inducedvertbreak',
            'vertbreak', 'platelocside', 'platelocheight', 'initposx', 'initposz']]
Y_RR = RHP_RHH['GIDP_SweetSpot']

X_train_RR, X_test_RR, y_train_RR, y_test_RR = train_test_split(
    np.array(X_RR), np.array(Y_RR),
    test_size=0.2,
    random_state=42,
    stratify=np.array(Y_RR)
)




In [10]:
print(X_train_RR.shape)  # If > 100k rows, try sampling first


(116162, 9)


In [11]:

#RHP v LHH
X_RL = RHP_LHH[['spinrate', 'relspeed', 'horzbreak', 'inducedvertbreak',
            'vertbreak', 'platelocside', 'platelocheight', 'initposx', 'initposz']]
Y_RL = RHP_LHH['GIDP_SweetSpot']

X_train_RL, X_test_RL, y_train_RL, y_test_RL = train_test_split(
    np.array(X_RL), np.array(Y_RL),
    test_size=0.2,
    random_state=42,
    stratify=np.array(Y_RL)
)

# Grid search for best hyperparameters
gam_RL = LogisticGAM(s(0) + s(1) + te(2, 3, 4) + 
                   te(5, 6) + te(7, 8)).gridsearch(X_train_RL, y_train_RL)

  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
KeyboardInterrupt



In [ ]:

#LHP v RHH
X_LR = LHP_RHH[['spinrate', 'relspeed', 'horzbreak', 'inducedvertbreak',
            'vertbreak', 'platelocside', 'platelocheight', 'initposx', 'initposz']]
Y_LR = LHP_RHH['GIDP_SweetSpot']

X_train_LR, X_test_LR, y_train_LR, y_test_LR = train_test_split(
    np.array(X_LR), np.array(Y_LR),
    test_size=0.2,
    random_state=42,
    stratify=np.array(Y_LR)
)

# Grid search for best hyperparameters
gam_LR = LogisticGAM(s(0) + s(1) + te(2, 3, 4) + 
                   te(5, 6) + te(7, 8)).gridsearch(X_train_LR, y_train_LR)
